In [1]:
from os import listdir,mkdir,rename
from os.path import isfile, join, isdir,exists,splitext
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames,process_sample_matrix,compute_distance_matrix_general


done1


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'01022018'

# load files:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFwithBDs'
SampleSpeciesDFwithBDs=pd.read_pickle(file1)


In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD'
SampleSpeciesDFgroupedByBD=pd.read_pickle(file2)

In [ ]:
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434'
SampleSpeciesDFgroupedByBD_only434=pd.read_pickle(file3)

In [ ]:
file4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/fullXgroupbyBD'
fullXgroupbyBD=pd.read_pickle(file4)

In [ ]:
file5='/net/mraid08/export/genie/Lab/Personal/ShaniBAF//TCR_real_data/PhenotypicData/fullXwithBDs'
fullXwithBDs=pd.read_pickle(file5)

In [ ]:
file6='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/fullXgroupbyBD_only434'
fullXgroupbyBD_only434=pd.read_pickle(file6)

In [ ]:
file7='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/binaryVariableList.txt'
with open(file7,'rb') as fp:
    binaryVariableList=pickle.load(fp)


# generate specific distance metrices

## distance matrix calculation function from Noam

In [ ]:
from scipy.spatial.distance import braycurtis, pdist

In [ ]:
def compute_distance_matrix(samps_by_features, distance_measure, do_binary = False):
    from scipy.spatial.distance import braycurtis, pdist
    A = samps_by_features.T.copy()
    temp_res = pd.DataFrame(1, index=A.columns, columns=A.columns, dtype=np.float64)
    A = A.values
    if do_binary:
        A[A>0] = 1
    for i in range(A.shape[1]):
        a = A[:,i]
        vals = []
        if i % 50 == 0:
            print i
        for j in range(A.shape[1]):
            b = A[:,j]
            val = None
            val = distance_measure(np.stack((a,b),axis=1).T)
            vals.append(val)
        temp_res.iloc[i,:] = vals
    return temp_res

## generate public sequence distance matrices

In [ ]:
# load the publicMore10 file:

file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan10_434Samples'
sharingMatrix_moreThan10_434Samples=pd.read_pickle(file3)

In [ ]:
#calculate distance matrix for the publicMore10 data:

temp_res=compute_distance_matrix(samps_by_features=sharingMatrix_moreThan10_434Samples, distance_measure=pdist, do_binary = False)

In [ ]:
PNP434_publicMore10_EucDistMat=temp_res

In [ ]:
# edit the sample names to match those in the phenotype files:

for n,sample in enumerate(PNP434_publicMore10_EucDistMat.index):
    if '_' in sample:
        NewName=sample.split('_')[0]
    else:
        NewName=sample
    if 'b' in NewName:
        print NewName
        NewName=NewName.split('b')[0]
        print NewName 
    PNP434_publicMore10_EucDistMat.rename(index={sample:NewName},inplace=True)
    PNP434_publicMore10_EucDistMat.rename(columns={sample:NewName},inplace=True)

In [ ]:
PNP434_publicMore10_EucDistMat.head()

In [ ]:
PNP434_publicMore10_EucDistMat=PNP434_publicMore10_EucDistMat.dropna(axis=(0,1),how='any')

In [ ]:
PNP434_publicMore10_EucDistMat

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_publicMore10_EucDistMat'
PNP434_publicMore10_EucDistMat.to_pickle(file1)

## generate age distance matrix:

In [ ]:
fullXgroupbyBD_only434.head()

In [ ]:
# calculate the distance matrix for age:

samps_by_features=pd.DataFrame(fullXgroupbyBD_only434['Age'])
distance_measure=pdist
do_binary=False
tempres=compute_distance_matrix(samps_by_features, distance_measure, do_binary = False)

In [ ]:
PNP434_Age_EucDistMat=tempres
PNP434_Age_EucDistMat.head()

In [ ]:
# remove nan rows/columns

PNP434_Age_EucDistMat=PNP434_Age_EucDistMat.dropna(axis=(0,1),how='all')

In [ ]:
PNP434_Age_EucDistMat

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_Age_EucDistMat'
PNP434_Age_EucDistMat.to_pickle(file1)

## Mantel test between age and publicMore10:

In [ ]:
from skbio.stats.distance import mantel

In [ ]:
x=PNP434_publicMore10_EucDistMat
y=PNP434_Age_EucDistMat


In [ ]:
print x.shape
print y.shape

for sample in x.index:
    if sample not in y.index:
        x=x.drop(sample,axis=0)
        x=x.drop(sample,axis=1)
for sample in y.index:
    if sample not in x.index:
        y=y.drop(sample,axis=0)
        y=y.drop(sample,axis=1)
        
print x.shape
print y.shape

In [ ]:

mantel(x, y, method='pearson', permutations=999, alternative='greater', strict=True, lookup=None)

## generate CRP distance matrix:

In [ ]:
# calculate the distance matrix for CRP:

samps_by_features=pd.DataFrame(fullXgroupbyBD_only434['CRP (WIDE RANGE)'])
distance_measure=pdist
do_binary=False
tempres=compute_distance_matrix(samps_by_features, distance_measure, do_binary = False)

In [ ]:
PNP434_CRPwide_EucDistMat=tempres
PNP434_CRPwide_EucDistMat.head()

In [ ]:
# remove nan rows/columns

PNP434_CRPwide_EucDistMat=PNP434_CRPwide_EucDistMat.dropna(axis=(0,1),how='all')

In [ ]:
PNP434_CRPwide_EucDistMat

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_CRPwide_EucDistMat'
PNP434_CRPwide_EucDistMat.to_pickle(file1)

## Mantel test between CRP and publicMore10:

In [ ]:
from skbio.stats.distance import mantel

In [ ]:
x=PNP434_publicMore10_EucDistMat
y=PNP434_CRPwide_EucDistMat


In [ ]:
print x.shape
print y.shape

for sample in x.index:
    if sample not in y.index:
        x=x.drop(sample,axis=0)
        x=x.drop(sample,axis=1)
for sample in y.index:
    if sample not in x.index:
        y=y.drop(sample,axis=0)
        y=y.drop(sample,axis=1)
        
print x.shape
print y.shape

In [ ]:

mantel(x, y, method='pearson', permutations=999, alternative='greater', strict=True, lookup=None)

## distance matrices for public sequence matrices:

In [ ]:
def gen_publicSeqMatrix(nShared_cutoff):
    
    # load  file:
    print 'loading data...'
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples' %nShared_cutoff
    sharingMatrix_moreThanN_434Samples=pd.read_pickle(file1)
    
    #calculate distance matrix for the publicMoreN data:
    print 'calculating distance matrix...'
    temp_res=compute_distance_matrix(samps_by_features=sharingMatrix_moreThanN_434Samples, distance_measure=pdist, do_binary = False)
    
    PNP434_publicMoreN_EucDistMat=temp_res
    
    # edit the sample names to match those in the phenotype files:
    print 'edit distance matrix...'
    for n,sample in enumerate(PNP434_publicMoreN_EucDistMat.index):
        if '_' in sample:
            NewName=sample.split('_')[0]
        else:
            NewName=sample
        if 'b' in NewName:
            NewName=NewName.split('b')[0]
        PNP434_publicMoreN_EucDistMat.rename(index={sample:NewName},inplace=True)
        PNP434_publicMoreN_EucDistMat.rename(columns={sample:NewName},inplace=True)
        
    PNP434_publicMoreN_EucDistMat=PNP434_publicMoreN_EucDistMat.dropna(axis=(0,1),how='any')
    
    # save resulting distance matrix to file:
    print 'saving to file...'
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_publicMore%s_EucDistMat' %nShared_cutoff
    PNP434_publicMoreN_EucDistMat.to_pickle(file2)

### calculate distance matrix for publicMoreN:
N=2/5/10

In [ ]:
gen_publicSeqMatrix(5)

In [ ]:
gen_publicSeqMatrix(10)

In [ ]:
gen_publicSeqMatrix(2)

### Calculate distance matrix for publicMoreN_relativeAbundance 
n=2/5/10

In [ ]:
def gen_publicSeqMatrix_RA(nShared_cutoff):
    
    # load  file:
    print 'loading data...'
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples_RA' %nShared_cutoff
    sharingMatrix_moreThanN_434Samples=pd.read_pickle(file1)
    
    #calculate distance matrix for the publicMoreN data:
    print 'calculating distance matrix...'
    temp_res=compute_distance_matrix(samps_by_features=sharingMatrix_moreThanN_434Samples, distance_measure=pdist, do_binary = False)
    
    PNP434_publicMoreN_EucDistMat=temp_res
    
    # edit the sample names to match those in the phenotype files:
    print 'edit distance matrix...'
    for n,sample in enumerate(PNP434_publicMoreN_EucDistMat.index):
        if '_' in sample:
            NewName=sample.split('_')[0]
        else:a
            NewName=sample
        if 'b' in NewName:
            NewName=NewName.split('b')[0]
        PNP434_publicMoreN_EucDistMat.rename(index={sample:NewName},inplace=True)
        PNP434_publicMoreN_EucDistMat.rename(columns={sample:NewName},inplace=True)
        
    PNP434_publicMoreN_EucDistMat=PNP434_publicMoreN_EucDistMat.dropna(axis=(0,1),how='any')
    
    # save resulting distance matrix to file:
    print 'saving to file...'
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_publicMore%s_RA_EucDistMat' %nShared_cutoff
    PNP434_publicMoreN_EucDistMat.to_pickle(file2)

In [ ]:
gen_publicSeqMatrix_RA(2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_publicMore10_RA_EucDistMat' 
PNP434_publicMore10_RA_EucDistMat=pd.read_pickle(file1)
PNP434_publicMore10_RA_EucDistMat.head()

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_publicMore10_EucDistMat' 
PNP434_publicMore10_EucDistMat=pd.read_pickle(file2)
PNP434_publicMore10_EucDistMat.head()

# general function for generating distance matrices

In [ ]:
'''
this function generates distance matrix for feature/phenotype matrices

matrix_name - free text to describe the feature/phenotype matrix (String)
matrix_df - matrix df. in the former version of the function I used the matrix_file as input and the function downloaded the matrix from the file
distance_measure = 'euclidean' for most cases, 'braycurtis' for 'ecological' matrices such as microbiome
species relative abundance or public TCR relative abundance
do_binary - True if necessary to convert the original matrix into a binary form
'''


def compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary):
    
    from scipy.spatial.distance import braycurtis, pdist
    
#     #load and process matrix file: ### I updated the function and now it gets as input the df itself and doesn't load it###
#     samps_by_features=pd.read_pickle(matrix_file)
    samps_by_features=matrix_df
    
    
    print 'original df shape is %s_%s' %(samps_by_features.shape[0],samps_by_features.shape[1])
    for column in samps_by_features.columns.values:    #remove nonnumeric columns
        if samps_by_features[column].dtype=='object':
            samps_by_features=samps_by_features.drop(column,axis=1)
            print '%s column was dropped from df' %column
    samps_by_features=samps_by_features.dropna(axis=(0,1),how='all')
    print 'df shape after processing is %s_%s' %(samps_by_features.shape[0],samps_by_features.shape[1])
    
    if do_binary:
        isBinary='binary'
    else:
        isBinary=''
    
    
    #calculate the distance matrix:
    print 'calculating distance matrix...'
    A = samps_by_features.T.copy()
    temp_res = pd.DataFrame(1, index=A.columns, columns=A.columns, dtype=np.float64)
    A = A.values
    if do_binary:
        A[A>0] = 1
    for i in range(A.shape[1]):
        a = A[:,i]
        vals = []
        if i % 50 == 0:
            print i
        for j in range(A.shape[1]):
            b = A[:,j]
            val = None
            val = pdist(np.stack((a,b),axis=1).T,distance_measure)
            vals.append(val)
        temp_res.iloc[i,:] = vals
        
    # edit the distance matrix:
    # (1) edit sample names to match those in the phenotype files
    # (2) remove rows and columns with nan values:
    print 'edit distance matrix...'
    for n,sample in enumerate(temp_res.index):
        if '_' in sample:
            NewName=sample.split('_')[0]
        else:
            NewName=sample
        if 'b' in NewName:
            NewName=NewName.split('b')[0]
        temp_res.rename(index={sample:NewName},inplace=True)
        temp_res.rename(columns={sample:NewName},inplace=True)
        
    print 'original distMat shape is %s_%s' %(temp_res.shape[0],temp_res.shape[1])
    bool_row_idx = temp_res.isnull().any(axis=0)
    row_idx_list=temp_res[bool_row_idx].index
    bool_col_idx = temp_res.isnull().any(axis=1)
    col_idx_list=temp_res[bool_col_idx].index
    for row_idx in row_idx_list:
        for col_idx in col_idx_list:
            print row_idx,col_idx
            print '%s row was dropped from distMat' %row_idx
            temp_res=temp_res.drop(row_idx,axis=0)
            print '%s column was dropped from distMat' %col_idx
            temp_res=temp_res.drop(col_idx,axis=1)
    print 'distMat shape after processing is %s_%s' %(temp_res.shape[0],temp_res.shape[1])
    
    # save resulting distance matrix to file:
    print 'saving to file...'
    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/%s%s' %(matrix_name, 
                                                          isBinary)  
    temp_res.to_pickle(file2)
        
    return temp_res

## Distance matrices for microbiome species matrices: 

### microbiomeSpecies_relativeAbndance bray curtis

In [ ]:
matrix_name='MicrobiomeSpeciesRA'
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434'
matrix_df=pd.read_pickle(matrix_file)
distance_measure='braycurtis'
do_binary = False

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary = False)

### microbiomeSpecies_relativeAbndance euclidean

In [ ]:
matrix_name='MicrobiomeSpeciesRA'
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434'
matrix_df=pd.read_pickle(matrix_file)
distance_measure='euclidean'
do_binary = False

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary = False)

### microbiomeSpecies_binary

In [ ]:
matrix_name='MicrobiomeSpecies'
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434'
matrix_df=pd.read_pickle(matrix_file)
distance_measure='euclidean'
do_binary = True

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary = do_binary)

### microbiomeSpecies_log2RA matrix:

In [ ]:
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434_log2scale'
matrix_name='MicrobiomeSpeciesRAlog2scale'
matrix_df=pd.read_pickle(matrix_file)
distance_measure='euclidean'
do_binary = False

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


## generate distance matrix for shared sequence matrices:

### RA matrices:

In [ ]:
n_cuttoff_list=[2,5,10]

for n,nShared_cutoff in enumerate(n_cuttoff_list):
    print n,nShared_cutoff
    matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples_RA' %nShared_cutoff
    matrix_df=pd.read_pickle(matrix_file)
    matrix_name='PNP434_sharingMatrix_moreThan%s_RA' %nShared_cutoff
    distance_measure='braycurtis'
    do_binary = False

    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary = do_binary)
    

In [ ]:
n_cuttoff_list=[50]

for n,nShared_cutoff in enumerate(n_cuttoff_list):
    print n,nShared_cutoff
    matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples_RA' %nShared_cutoff
    matrix_df=pd.read_pickle(matrix_file)
    matrix_name='PNP434_sharingMatrix_moreThan%s_RA' %nShared_cutoff
    distance_measure='braycurtis'
    do_binary = False

    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary = do_binary)
    

### shared sequences more than 1:

In [ ]:
#shared more than 1, RA, bray curtis:
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan1_434Samples_RA'
matrix_df=pd.read_pickle(matrix_file)
matrix_name='PNP434_sharingMatrix_moreThan1_RA'
distance_measure='braycurtis'
do_binary = False

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


In [ ]:
#shared more than 1, binary, bray curtis:
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan1_434Samples_RA'
matrix_df=pd.read_pickle(matrix_file)
matrix_name='PNP434_sharingMatrix_moreThan1'
distance_measure='braycurtis'
do_binary = True

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


In [ ]:
#shared more than 1, binary, euclidean:
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan1_434Samples_RA'
matrix_df=pd.read_pickle(matrix_file)
matrix_name='PNP434_sharingMatrix_moreThan1'
distance_measure='euclidean'
do_binary = True

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


In [ ]:
#shared more than 1, log2scale RA, euclidean:
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan1_434Samples_RA_log2scale'
matrix_df=pd.read_pickle(matrix_file)
matrix_name='PNP434_sharingMatrix_moreThan1_RA_log2scale'
distance_measure='euclidean'
do_binary = False

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


### more than 50, binary

In [ ]:
#shared more than 50, binary, euclidean:
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan50_434Samples_RA'
matrix_df=pd.read_pickle(matrix_file)
matrix_name='distMat_PNP434_sharingMatrix_moreThan50'
distance_measure='euclidean'
do_binary = True

compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


### jaccard distances:

In [4]:
nCutOffList=[50,10,1]

for nCutOff in nCutOffList:
    print nCutOff
    matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples' %nCutOff
    matrix_df=pd.read_pickle(matrix_file)
    matrix_name='sharingMatrixMoreThan50%s' %nCutOff
    distance_measure='jaccard'
    do_binary = False

    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


50
original df shape is 434_968
df shape after processing is 434_968
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processing is 434_434
saving to file...
10
original df shape is 434_19477
df shape after processing is 434_19477
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processing is 434_434
saving to file...
1
original df shape is 434_437132
df shape after processing is 434_437132
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processing is 434_434
saving to file...


## generate distance matrices for shared sequences PCs matrices:

### more than 1, use different numbers of PCs:

In [ ]:
nSharedCutoff=1
nPClist=[1,5,50,100]
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples_100PCs' %nSharedCutoff
for n in nPClist:
    print n
    
    matrix_df=pd.read_pickle(matrix_file)
    matrix_df=matrix_df.iloc[:,:n]
    print matrix_df.shape
    matrix_name='sharingMatrixMoreThan%s%sPCsUsed' %(nSharedCutoff,n)
    distance_measure='euclidean'
    do_binary = False

    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)

### more than 10, use different numbers of PCs:

In [ ]:
nSharedCutoff=10
nPClist=[1,5,50,100]
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples_100PCs' %nSharedCutoff
for n in nPClist:
    print n
    
    matrix_df=pd.read_pickle(matrix_file)
    matrix_df=matrix_df.iloc[:,:n]
    print matrix_df.shape
    matrix_name='sharingMatrixMoreThan%s%sPCsUsed' %(nSharedCutoff,n)
    distance_measure='euclidean'
    do_binary = False

    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)

### more than 50, use different numbers of PCs:

In [5]:
nSharedCutoff=50
nPClist=[1,5,50,100]
matrix_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples_100PCs' %nSharedCutoff
for n in nPClist:
    print n
    
    matrix_df=pd.read_pickle(matrix_file)
    matrix_df=matrix_df.iloc[:,:n]
    print matrix_df.shape
    matrix_name='sharingMatrixMoreThan%s%sPCsUsed' %(nSharedCutoff,n)
    distance_measure='euclidean'
    do_binary = False

    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)

1
(434, 1)
original df shape is 434_1
df shape after processing is 434_1
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processing is 434_434
saving to file...
5
(434, 5)
original df shape is 434_5
df shape after processing is 434_5
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processing is 434_434
saving to file...
50
(434, 50)
original df shape is 434_50
df shape after processing is 434_50
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processing is 434_434
saving to file...
100
(434, 100)
original df shape is 434_100
df shape after processing is 434_100
calculating distance matrix...
0
50
100
150
200
250
300
350
400
edit distance matrix...
original distMat shape is 434_434
distMat shape after processi

## generate distance matrices for variuos phenotypes from the newPhenotypeTable_groupedByBD file:



In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/columnsToAnalyzeFromNewFile'
with open(file1,'rb') as fp:
    columnsToAnalyzeFromNewFile=pickle.load(fp)

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/newPhenotypeTable_groupedByBD_only434'
newPhenotypeTable_groupedByBD_only434=pd.read_pickle(file2)

In [ ]:
distance_measure='euclidean'
do_binary=False

for n,column in enumerate(columnsToAnalyzeFromNewFile):
    print n,column
    matrix_df=pd.DataFrame(newPhenotypeTable_groupedByBD_only434[column])
#     print matrix_df
    matrix_name='PNP434_%s_EucDistMat' %column
    compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)
    

## generate distance matrix for binary phenotype:

In [ ]:
distance_measure='euclidean'
do_binary=False

column='Gender'
matrix_df=pd.DataFrame(fullXgroupbyBD[column])
matrix_name='distMat_PNP434_%s_euclidean' %column
temp_res=compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)


In [ ]:
temp_res.head()

In [ ]:
#get binary variable list:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/binaryVariableList.txt'

with open(file1,'rb') as fp:
    binaryVariableList=pickle.load(fp)


In [ ]:
distance_measure='euclidean'
do_binary=False

for n,phenotype in enumerate(binaryVariableList):
    print n,phenotype
    matrix_df=pd.DataFrame(fullXgroupbyBD[phenotype])
    matrix_name= phenotype
    temp_res=compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)




### rename newPhenotype distance files:

In [ ]:
##### USE WITH CAUTIUOS! ####


path =  '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes'
filenames = os.listdir(path)

for f in filenames:
    filename, file_extension = os.path.splitext(f)
    phenotype=filename.split('_')[1]
    newFileName='distMat_PNP434_%s' %phenotype
#     print newFileName
#     print phenotype
    os.rename(os.path.join(path, f),os.path.join(path, newFileName))


In [ ]:
##### USE WITH CAUTIUOS! ####

path =  '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes'
filenames = os.listdir(path)

for f in filenames:
    filename, file_extension = os.path.splitext(f)
    if '_euclidean' in filename:
        newFileName=filename
    else:
        newFileName=filename+'_euclidean'
#     print newFileName
#     print phenotype
    os.rename(os.path.join(path, f),os.path.join(path, newFileName))


In [ ]:
##### USE WITH CAUTIUOS! ####


path =  '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes'
filenames = os.listdir(path)

for f in filenames:
    filename, file_extension = os.path.splitext(f)
    phenotype=filename.split('_')[1]
    newFileName='distMat_PNP434_%s_euclidean' %phenotype
#     print filename
#     print newFileName

    os.rename(os.path.join(path, f),os.path.join(path, newFileName))


In [ ]:
##### USE WITH CAUTIUOS! ####


path =  '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/NewFeatures'
filenames = listdir(path)

for n,f in enumerate(filenames):
#     if n<10:
    
        filename, file_extension = splitext(f)
        splitted=filename.split('_')
        newName=splitted[2:-1]
        myString = "_".join(newName)

#         print splitted, myString
    
#     newFileName='distMat_PNP434_%s_euclidean' %phenotype
#     print filename
#     print newFileName

#         rename(join(path, f),join(path, myString))


## generate distance matrices for more features:

### prepare the files:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/featureSummaryDFs/PNPsamples_434_allFeatures'
PNPsamples_434_allFeatures=pd.read_pickle(file1)
PNPsamples_434_allFeatures.head()

In [ ]:
from Feature_phenotype_functions import editSampleNames

In [ ]:
PNPsamples_434_allFeatures_edited=editSampleNames(PNPsamples_434_allFeatures)
PNPsamples_434_allFeatures_edited.head()

### check which features are highly correlated

In [ ]:
generalFeatureNames=[]
for column in PNPsamples_434_allFeatures_edited.columns.values:
    if not 'percPublic' in column:
        general=column.replace('_T','')
        general=general.replace('_0','')
        general=general.replace('_1','')
        generalFeatureNames.append(general)
print len(generalFeatureNames)

# from collections import Counter
# Counter(generalFeatureNames)

    

In [ ]:
generalFeatureNames=list(set(generalFeatureNames))
print len(generalFeatureNames)


In [ ]:
corrDF=pd.DataFrame()
for i in range(len(generalFeatureNames)):
    print i
    colT=generalFeatureNames[i]+'_T'
    col0=generalFeatureNames[i]+'_0'
    col1=generalFeatureNames[i]+'_1'
    
    if (colT in PNPsamples_434_allFeatures_edited.columns.values) and (col0 in PNPsamples_434_allFeatures_edited.columns.values) and (col1 in PNPsamples_434_allFeatures_edited.columns.values):  
        r_T0,p_T0=MyPearsonr(PNPsamples_434_allFeatures_edited[colT],PNPsamples_434_allFeatures_edited[col0])
        r_T1,p_T1=MyPearsonr(PNPsamples_434_allFeatures_edited[colT],PNPsamples_434_allFeatures_edited[col1])
        r_10,p_10=MyPearsonr(PNPsamples_434_allFeatures_edited[col0],PNPsamples_434_allFeatures_edited[col1])
        corrDF.loc[generalFeatureNames[i],'r_T0']=r_T0
        corrDF.loc[generalFeatureNames[i],'r_T1']=r_T1
        corrDF.loc[generalFeatureNames[i],'r_10']=r_10
    #     print colT
    #     print r_T0,r_T1,r_10
    
    for j in range(i,len(generalFeatureNames)):
        colJ=generalFeatureNames[j]+'_T'
        if (colT in PNPsamples_434_allFeatures_edited.columns.values) and (colJ in PNPsamples_434_allFeatures_edited.columns.values):
            r_TJ,p_TJ=MyPearsonr(PNPsamples_434_allFeatures_edited[colT],PNPsamples_434_allFeatures_edited[colJ])
            corrDF.loc[generalFeatureNames[i],generalFeatureNames[j]]=r_TJ
#         print colJ, r_TJ
        
        

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/featureSummaryDFs/PNPsamples_434_allFeatures_correlations'
corrDF.to_pickle(file1)

In [ ]:
corrDF.sort_values(by='r_T0').head()

### generate a list of features for which I need to calculate distance matrices:

In [ ]:
# prod and non prod features which are needed in addition to their corresponding total feature:
prodFeaturesToAdd=[]
nonProdFeaturesToAdd=[]

for n in corrDF.index:
    if corrDF.loc[n,'r_T0']<0.5:
        nonProdFeature=n+'_0'
        nonProdFeaturesToAdd.append(nonProdFeature)
    if corrDF.loc[n,'r_T1']<0.5:
        prodFeature=n+'_1'
        prodFeaturesToAdd.append(prodFeature)

print len(prodFeaturesToAdd)
print len(nonProdFeaturesToAdd)
        
        
    

In [ ]:
#total features that should be removed as they are highly correlated with another total feature:

columnList=corrDF.columns.values[3:]

totalFeaturesToRemove=[]

for n in corrDF.index:
    for col in columnList:
        if (corrDF.loc[n,col]>0.9) and (n!=col) and (n not in totalFeaturesToRemove) :
            totalFeaturesToRemove.append(col)
            print n,col, corrDF.loc[n,col]



In [ ]:
print len(generalFeatureNames)
print len(totalFeaturesToRemove)
print len(generalFeatureNames)-len(totalFeaturesToRemove)
totalFeatures=[x+'_T' for x in generalFeatureNames if x not in totalFeaturesToRemove]
print len(totalFeatures)

In [ ]:
print totalFeatures[:5]

In [ ]:
totalFeatures=totalFeatures+prodFeaturesToAdd+nonProdFeaturesToAdd
print len(totalFeatures)

In [ ]:
'cdr3Length_mean_T' in totalFeatures

In [ ]:
totalFeatures

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/featureSummaryDFs/totalFeatures','wb') as fp:
    pickle.dump(totalFeatures,fp)

### generate distance matrices for all features in the TotalFeatures list:

In [ ]:
distance_measure='euclidean'
do_binary=False

for n,column in enumerate(totalFeatures):
    if n>142:
        print n,column
        matrix_df=pd.DataFrame(PNPsamples_434_allFeatures_edited[column])
        matrix_name='distMat_PNP434_%s_%s' %(column, distance_measure)
        compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)
    

features from the list that generated an error:
n=52, 75,94,100,141,142

### generate lists for featureCombinationDistanceMatrices:

In [ ]:
LengthFeatures=['n1Insertion','n2Insertion','vDeletion','d3Deletion','d5Deletion','jDeletion','cdr3Length']
LengthFeaturesToUse=[]

for f in LengthFeatures:
    for feature in totalFeatures:
        if f in feature:
            LengthFeaturesToUse.append(feature)

print len(LengthFeaturesToUse)
print LengthFeaturesToUse
            

In [ ]:
ClonalityFeatures=['meanClonal_nt_T','meanClonal_aa_T','topClonal_nt_T','topClonal_aa_T','frequencyCount (%)_mean_T',
                         'frequencyCount (%)_max_T','shannon_nt_T','simpson_nt_T','berger_nt_T','shannon_aa_T','simpson_aa_T','berger_aa_T']

ClonalityFeaturesToUse=[]

for f in ClonalityFeatures:
    for feature in totalFeatures:
        if f in feature:
            ClonalityFeaturesToUse.append(feature)

print len(ClonalityFeaturesToUse)
print ClonalityFeaturesToUse[:5]




In [ ]:
regex_vFamily=re.compile('V.._.')
vFamily=[n for n in PNPsamples_434_allFeatures_edited.columns.values if re.match(regex_vFamily,n) ]

regex_vGene=re.compile('V..-.._.')
vGene=[n for n in PNPsamples_434_allFeatures_edited.columns.values if re.match(regex_vGene,n) ]

regex_jGene=re.compile('J..-.._.')
jGene=[n for n in PNPsamples_434_allFeatures_edited.columns.values if re.match(regex_jGene,n) ]

regex_dFamily=re.compile('D.._.')
dFamily=[n for n in PNPsamples_434_allFeatures_edited.columns.values if re.match(regex_dFamily,n) ]

regex_VJ=re.compile('V.._J.._.')
VJ=[n for n in PNPsamples_434_allFeatures_edited.columns.values if re.match(regex_VJ,n) ]

regex_DJ=re.compile('D.._J..-.._.')
DJ=[n for n in PNPsamples_434_allFeatures_edited.columns.values if re.match(regex_DJ,n) ]

usageFeatures=vFamily+vGene+jGene+dFamily+VJ+DJ
print len(usageFeatures)

usageFeaturesToUse=[]

for f in usageFeatures:
    for feature in totalFeatures:
        if f in feature:
            usageFeaturesToUse.append(feature)
            usageFeaturesToUse=list(set(usageFeaturesToUse))

print len(usageFeaturesToUse)
print usageFeaturesToUse


In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/featureSummaryDFs/LengthFeaturesToUse','wb') as fp:
    pickle.dump(LengthFeaturesToUse,fp)
    
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/featureSummaryDFs/ClonalityFeatures','wb') as fp:
    pickle.dump(ClonalityFeaturesToUse,fp)

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/featureSummaryDFs/usageFeaturesToUse','wb') as fp:
    pickle.dump(usageFeaturesToUse,fp)

### generate distance matrices for feature Combination:

In [ ]:
featureCombinations=[LengthFeaturesToUse,ClonalityFeaturesToUse,usageFeaturesToUse]
featureCombinationsNames=['LengthFeatures','ClonalityFeatures','usageFeatures']
distance_measure='euclidean'
do_binary=False

for n,combination in enumerate(featureCombinations):
    if n==2:
        n,combination
        combinationName=featureCombinationsNames[n]
        matrix_df=PNPsamples_434_allFeatures_edited[combination]
        if combination==usageFeaturesToUse:
            matrix_df=matrix_df.fillna(0)
        matrix_name='distMat_PNP434_%s_%s' %(combinationName, distance_measure)
        compute_distance_matrix_general(matrix_name,matrix_df, distance_measure, do_binary)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/distMat_PNP434_distMat_PNP434_ClonalityFeatures_euclidean_euclidean'
distMat_PNP434_distMat_PNP434_ClonalityFeatures_euclidean_euclidean=pd.read_pickle(file1)
distMat_PNP434_distMat_PNP434_ClonalityFeatures_euclidean_euclidean.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/distMat_PNP434_distMat_PNP434_usageFeatures_euclidean_euclidean'
distMat_PNP434_distMat_PNP434_usageFeatures_euclidean_euclidean=pd.read_pickle(file1)
distMat_PNP434_distMat_PNP434_usageFeatures_euclidean_euclidean.head()

# run mantel test for different combinations of features and phenotypes and summarize the results:

## define function to run matel test for different distance matrix combinations:

In [ ]:
def reject_outliers(data, m):
    outlier_ind=abs(data - np.mean(data)) < m * np.std(data)
    return outlier_ind

In [ ]:
'''
genPlot - True/False



'''


def mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative,removeSameUser):
    
    from skbio.stats.distance import mantel
    
    MantelFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelTestResults'
    MantelName='MantelDF_%s_%s_%s_%s_%s_removeSameUser%s' %(feature_name,phenotype_name, method,n_permut,alternative,removeSameUser)
    MantelFile='%s/%s' %(MantelFolder,MantelName)
    existingMantels=[f for f in listdir( MantelFolder) if isfile(join(MantelFolder, f))]
    
    if MantelName not in existingMantels:
    
        #(1)load and process distance matrix files:
        print 'loading and processing distance matrix files...'
        feature_dist_mat=pd.read_pickle(feature_dist_file)
        phenotype_dist_mat=pd.read_pickle(phenotype_dist_file)

        x=feature_dist_mat
        y=phenotype_dist_mat

        print 'original x array shape is %s_%s' %(x.shape[0],x.shape[1])
        print 'original y array shape is %s_%s' %(y.shape[0],y.shape[1])

        for sample in x.index:
            if sample not in y.index:
                x=x.drop(sample,axis=0)
                x=x.drop(sample,axis=1)
        for sample in y.index:
            if sample not in x.index:
                y=y.drop(sample,axis=0)
                y=y.drop(sample,axis=1)
                
        for column in x:    #remove nonnumeric columns
            if x[column].dtype=='object':
                x=x.drop(column,axis=1)
                print '%s column was dropped from x' %column
        x=x.dropna(axis=(0,1),how='all') #remove empty rows
        
        for column in y:    #remove nonnumeric columns
            if y[column].dtype=='object':
                y=y.drop(column,axis=1)
                print '%s column was dropped from y' %column
        y=y.dropna(axis=(0,1),how='all') #remove empty rows

        
        #make sure both matrices have the same order:
        x=x.sort_index(axis=0)
        x=x.sort_index(axis=1)
        y=y.sort_index(axis=0)
        y=y.sort_index(axis=1)
        
        print x.head()
        print y.head()
        
        if removeSameUser: #Remove second sample from the same user:
            print 'removing second sample from the same user...'
            removedSampleList=['BD714','BD838'] #UPDATE LIST WITH MORE SAMPLES COMING FROM AN ALREADY EXIST USER
            for removedSample in removedSampleList:
                x=x.drop(removedSample,axis=0)
                x=x.drop(removedSample,axis=1)
                y=y.drop(removedSample,axis=0)
                y=y.drop(removedSample,axis=1)
        
        print 'new x array shape is %s_%s' %(x.shape[0],x.shape[1])
        print 'new y array shape is %s_%s' %(y.shape[0],y.shape[1])
        print x.head()
        print y.head()
            
#         # to reject outliers, fix the following lines, add stdToReject as input variable and uncomment:
#         if stdToReject is not None:
#             nx_outliers=reject_outliers(newx, m=stdToReject)
#             ny_outliers=reject_outliers(newy, m=stdToReject)
#             n_outliers=nx_outliers+ny_outliers
#             finalx=newx[~n_outliers]
#             finaly=newy[~n_outliers]
#         else:
#             finalx=newx
#             finaly=newy
        
        
        #(2) run mantel:
        print 'running mantel test...'
        corr_coeff,p_value,N = mantel(x, y, method=method, permutations=n_permut, alternative=alternative, strict=True, lookup=None)

        mantelDF=pd.DataFrame()
        mantelDF.loc[0,'feature_name']=feature_name
        mantelDF.loc[0,'phenotype_name']=phenotype_name
        mantelDF.loc[0,'method']=method
        mantelDF.loc[0,'n_permut']=n_permut
        mantelDF.loc[0,'alternative']=alternative
        mantelDF.loc[0,'corr_coeff']=corr_coeff
        mantelDF.loc[0,'p_value']=p_value
        mantelDF.loc[0,'feature_name']=feature_name
        mantelDF.loc[0,'N']=N
        
#         mantelDF.to_pickle(MantelFile)
        print p_value
        
        
#         #generate correlation plot if desired:
#         if p_value<0.05 or corr_coeff>0.15:
#             fig1=plot_corr_for_distMats(x,y,feature_name,phenotype_name)
#             file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/DistMat_correlation_plots/%s_%s' %(feature_name,phenotype_name)
#             fig1.savefig(file1,dpi=200)
            
        
        
        print 'DONE!'
    
    else:
        print 'This Mantel Test result already exist in folder'
    

## check distance matrices order and the effect on the mantel test:

### Run without ordering the distance matrices:

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'

fileNameP='distMat_PNP434_Age_euclidean'
fileNameF='distMat_PNP434_sharingMatrixMoreThan1RA_braycurtis'

feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
feature_name=fileNameF.split('_')[-2]+'_'+fileNameF.split('_')[-1]

phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/%s' %fileNameP
phenotype_name=fileNameP.split('_')[-2]+'_'+fileNameP.split('_')[-1]


               

mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative)

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'

fileNameP='distMat_PNP434_Albumin_euclidean'
fileNameF='distMat_PNP434_sharingMatrixMoreThan1RA_braycurtis'

feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
feature_name=fileNameF.split('_')[-2]+'_'+fileNameF.split('_')[-1]

phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/%s' %fileNameP
phenotype_name=fileNameP.split('_')[-2]+'_'+fileNameP.split('_')[-1]


               

mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative)

### run with ordering the distance matrix

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'

fileNameP='distMat_PNP434_Age_euclidean'
fileNameF='distMat_PNP434_sharingMatrixMoreThan1RA_braycurtis'

feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
feature_name=fileNameF.split('_')[-2]+'_'+fileNameF.split('_')[-1]

phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/%s' %fileNameP
phenotype_name=fileNameP.split('_')[-2]+'_'+fileNameP.split('_')[-1]


               

mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative)

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'

fileNameP='distMat_PNP434_Albumin_euclidean'
fileNameF='distMat_PNP434_sharingMatrixMoreThan1RA_braycurtis'

feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
feature_name=fileNameF.split('_')[-2]+'_'+fileNameF.split('_')[-1]

phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/%s' %fileNameP
phenotype_name=fileNameP.split('_')[-2]+'_'+fileNameP.split('_')[-1]


               

mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative)

### remove samples from the same user:

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'

fileNameP='distMat_PNP434_Age_euclidean'
fileNameF='distMat_PNP434_sharingMatrixMoreThan1RA_braycurtis'

feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
feature_name=fileNameF.split('_')[-2]+'_'+fileNameF.split('_')[-1]

phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/%s' %fileNameP
phenotype_name=fileNameP.split('_')[-2]+'_'+fileNameP.split('_')[-1]


               

mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative,removeSameUser=True)

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'

fileNameP='distMat_PNP434_Albumin_euclidean'
fileNameF='distMat_PNP434_sharingMatrixMoreThan1RA_braycurtis'

feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
feature_name=fileNameF.split('_')[-2]+'_'+fileNameF.split('_')[-1]

phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/%s' %fileNameP
phenotype_name=fileNameP.split('_')[-2]+'_'+fileNameP.split('_')[-1]


               

mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative,removeSameUser=True)

## running tests:

In [ ]:
PhenotypeDistFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes'
phenotypeFiles = [f for f in listdir(PhenotypeDistFolder) if isfile(join(PhenotypeDistFolder, f))]
FeatureDistFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features'
featureFiles = [f for f in listdir(FeatureDistFolder) if isfile(join(FeatureDistFolder, f))]

In [ ]:
featureFiles

In [ ]:
phenotypeFiles

In [ ]:
method='spearman'
n_permut=9999
alternative='two-sided'


for i,fileNameF in enumerate(featureFiles):
    if i<2:
        feature_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/%s' %fileNameF
### update name and uncomment!!        feature_name=fileNameF.split('_')[1]+fileNameF.split('_')[2]+fileNameF.split('_')[3]
#         print i,feature_name

        for j,fileNameP in enumerate(phenotypeFiles):
#             if j<4:
                phenotype_dist_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes/%s' %fileNameP
### update name and uncomment!!                phenotype_name=fileNameP.split('_')[1]+fileNameP.split('_')[2]
                print i, j, feature_name, phenotype_name
               

                mantelTest_feature_phenotype(feature_dist_file, phenotype_dist_file,feature_name,phenotype_name, method,n_permut,alternative)
    

### run again for new phenotypes and new features - 070118

on eclipse!!

### check which old phenotypes should I ran:

In [ ]:
pathOLD =  '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes'
filenamesOLD = os.listdir(pathOLD)

In [ ]:
pathNEW =  '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes'
filenamesNEW = os.listdir(pathNEW)

In [ ]:
for f in filenamesOLD:
    if f not in filenamesNEW:
        print f

In [ ]:
for f in filenamesNEW:
    if 'TSH' in f:
        print f

*** CONCLUSIONS: ***
the only phenotypes that exist in the old phenotype folder and not in the new one are:
MPV, WHR, BPdia, BPsys, Platelets

I moved them to the new phenotype folder  - 7/1/18

In [ ]:
# number of mantel tests - 7.1.18 = 1260
70*18

In [ ]:
# corrected p-value - 7.1.18 - 0.00004
0.05/1260

## view mantel tests results - 'two sided:

In [ ]:
MantelFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/NewMantelTestResults'
Mantels=concat_summarizing_dfs(MantelFolder)

In [ ]:
Mantels.sort_values(by='p_value')

In [ ]:
Mantels.sort_values(by='corr_coeff',ascending=False)

In [ ]:
Mantels[Mantels['phenotype_name'].str.contains('Mic')].sort_values(by='p_value')

In [ ]:
Mantels['feature_name'].unique()

In [ ]:
featureList=Mantels['feature_name'].unique()
print len(featuresList)

In [ ]:
for i, feature in enumerate(featureList):
    if i==0:
#         print i
        MantelOrganizedResults=Mantels[Mantels['feature_name']==feature]
#         print len(MantelOrganizedDF)
    else:
        MantelOrganizedResults=pd.merge(MantelOrganizedResults,Mantels[Mantels['feature_name']==feature],
                                  how='outer', left_on=['phenotype_name','method','n_permut', 'alternative'],
                                               right_on=['phenotype_name','method','n_permut', 'alternative'],
                                    suffixes=('','_%s' %feature))
#         print len(MantelOrganizedDF) 

MantelOrganizedResults=MantelOrganizedResults.rename(columns={'corr_coeff':'corr_coeff_sharingMatrixMoreThan101PCsUsed_euclidean',
                                                    'p_value':'p_value_sharingMatrixMoreThan101PCsUsed_euclidean',
                                                    'N':'N_sharingMatrixMoreThan101PCsUsed_euclidean'})
for column in MantelOrganizedResults.columns.values:
        if 'feature_name' in column:
            MantelOrganizedResults=MantelOrganizedResults.drop(column,axis=1)
        newColName=column.replace('sharingMatrixMoreThan','')
#         newColName=newColName.replace('_euclidean','')
#         newColName=newColName.replace('_braycurtis','')
        MantelOrganizedResults=MantelOrganizedResults.rename(columns={column:newColName})
                                                    
MantelOrganizedResults.head()       
    

In [ ]:
pList=[]
rList=[]

for column in MantelOrganizedResults.columns.values:
    if 'p_value' in column:
        pList.append(column)
    if 'corr_coeff' in column:
        rList.append(column)

MantelOrganizedResults['Mean p_value']=MantelOrganizedResults[pList].mean(axis=1)
MantelOrganizedResults['Mean corr_coeff']=MantelOrganizedResults[rList].mean(axis=1)
MantelOrganizedResults.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults'
MantelOrganizedResults.to_pickle(file1)

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults.xlsx'
MantelOrganizedResults.to_excel(file1)

In [ ]:
MantelOrganizedResults.columns.values

In [ ]:
for column in MantelOrganizedResults.columns.values:
    if 'Binary' in column:
        print column

## view mantel tests results - 'greater:

In [ ]:
MantelFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/NewMantelTestResults_greater'
Mantels_greater=concat_summarizing_dfs(MantelFolder)

In [ ]:
Mantels_greater.sort_values(by='p_value')

In [ ]:
Mantels_greater.sort_values(by='corr_coeff',ascending=False)

In [ ]:
Mantels_greater[Mantels_greater['phenotype_name'].str.contains('Mic')].sort_values(by='p_value')

In [ ]:
Mantels_greater['feature_name'].unique()

In [ ]:
featureList=Mantels_greater['feature_name'].unique()
print len(featuresList)

In [ ]:
for i, feature in enumerate(featureList):
    if i==0:
#         print i
        MantelOrganizedResults_greater=Mantels_greater[Mantels_greater['feature_name']==feature]
#         print len(MantelOrganizedDF)
    else:
        MantelOrganizedResults_greater=pd.merge(MantelOrganizedResults_greater,Mantels_greater[Mantels_greater['feature_name']==feature],
                                  how='outer', left_on=['phenotype_name','method','n_permut', 'alternative'],
                                               right_on=['phenotype_name','method','n_permut', 'alternative'],
                                    suffixes=('','_%s' %feature))
#         print len(MantelOrganizedResults_greater) 

MantelOrganizedResults_greater=MantelOrganizedResults_greater.rename(columns={'corr_coeff':'corr_coeff_sharingMatrixMoreThan101PCsUsed_euclidean',
                                                    'p_value':'p_value_sharingMatrixMoreThan101PCsUsed_euclidean',
                                                    'N':'N_sharingMatrixMoreThan101PCsUsed_euclidean'})
for column in MantelOrganizedResults_greater.columns.values:
        if 'feature_name' in column:
            MantelOrganizedResults_greater=MantelOrganizedResults_greater.drop(column,axis=1)
        newColName=column.replace('sharingMatrixMoreThan','')
#         newColName=newColName.replace('_euclidean','')
#         newColName=newColName.replace('_braycurtis','')
        MantelOrganizedResults_greater=MantelOrganizedResults_greater.rename(columns={column:newColName})
                                                    
MantelOrganizedResults_greater.head()       
    

In [ ]:
pList=[]
rList=[]

for column in MantelOrganizedResults_greater.columns.values:
    if 'p_value' in column:
        pList.append(column)
    if 'corr_coeff' in column:
        rList.append(column)

MantelOrganizedResults_greater['Mean p_value']=MantelOrganizedResults_greater[pList].mean(axis=1)
MantelOrganizedResults_greater['Mean corr_coeff']=MantelOrganizedResults_greater[rList].mean(axis=1)
MantelOrganizedResults_greater['Min p_value']=MantelOrganizedResults_greater[pList].min(axis=1)
MantelOrganizedResults_greater['Max corr_coeff']=MantelOrganizedResults_greater[rList].max(axis=1)
MantelOrganizedResults_greater.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults_greater'
MantelOrganizedResults_greater.to_pickle(file1)

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults_greater.xlsx'
MantelOrganizedResults_greater.to_excel(file1)

In [ ]:
MantelOrganizedResults.columns.values

In [ ]:
for column in MantelOrganizedResults.columns.values:
    if 'Binary' in column:
        print column

## view mantel tests results - 'greater_100000:

In [ ]:
MantelFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/NewMantelTestResults_greater_100000'
Mantels_greater_100000=concat_summarizing_dfs(MantelFolder)

In [ ]:
0.05/1260

In [ ]:
Mantels_greater_100000.sort_values(by='p_value')

In [ ]:
Mantels_greater_100000.sort_values(by='corr_coeff',ascending=False)

In [ ]:
Mantels_greater_100000[Mantels_greater_100000['phenotype_name'].str.contains('Mic')].sort_values(by='p_value')

In [ ]:
Mantels_greater_100000['feature_name'].unique()

In [ ]:
print len(Mantels_greater_100000['feature_name'].unique())

In [ ]:
featureList=Mantels_greater_100000['feature_name'].unique()

In [ ]:
for i, feature in enumerate(featureList):
    print i, feature
    if i==0:

        MantelOrganizedResults_greater100000=Mantels_greater_100000[Mantels_greater_100000['feature_name']==feature]

    else:
        MantelOrganizedResults_greater100000=pd.merge(MantelOrganizedResults_greater100000,Mantels_greater_100000[Mantels_greater_100000['feature_name']==feature],
                                  how='outer', left_on=['phenotype_name','method','n_permut', 'alternative'],
                                               right_on=['phenotype_name','method','n_permut', 'alternative'],
                                    suffixes=('','_%s' %feature))


MantelOrganizedResults_greater100000=MantelOrganizedResults_greater100000.rename(columns={'corr_coeff':'corr_coeff_sharingMatrixMoreThan150PCsUsed_euclidean',
                                                    'p_value':'p_value_sharingMatrixMoreThan150PCsUsed_euclidean',
                                                    'N':'N_sharingMatrixMoreThan150PCsUsed_euclidean'})
for column in MantelOrganizedResults_greater100000.columns.values:
        if 'feature_name' in column:
            MantelOrganizedResults_greater100000=MantelOrganizedResults_greater100000.drop(column,axis=1)
        newColName=column.replace('sharingMatrixMoreThan','')
        MantelOrganizedResults_greater100000=MantelOrganizedResults_greater100000.rename(columns={column:newColName})
                                                    
MantelOrganizedResults_greater100000.head()       
    

In [ ]:
pList=[]
rList=[]

for column in MantelOrganizedResults_greater100000.columns.values:
    if 'p_value' in column:
        pList.append(column)
    if 'corr_coeff' in column:
        rList.append(column)

MantelOrganizedResults_greater100000['Mean p_value']=MantelOrganizedResults_greater100000[pList].mean(axis=1)
MantelOrganizedResults_greater100000['Mean corr_coeff']=MantelOrganizedResults_greater100000[rList].mean(axis=1)
MantelOrganizedResults_greater100000['Min p_value']=MantelOrganizedResults_greater100000[pList].min(axis=1)
MantelOrganizedResults_greater100000['Max corr_coeff']=MantelOrganizedResults_greater100000[rList].max(axis=1)
MantelOrganizedResults_greater100000.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults_greater100000'
MantelOrganizedResults_greater100000.to_pickle(file1)

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults_greater100000.xlsx'
MantelOrganizedResults_greater100000.to_excel(file1)

## running again mantel tests: for new features - 24.1.18

This was ran in eclipse using runMantel2:
129 feature distance matrices and 70 phenotype distance matrices
in total: 9030 tests
required p-value=5.5*10-6
minimal p-value with 99999 permutations=10-5

In [ ]:
MantelFolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/NewMantelTestResults_newFeatures'
Mantels_newFeatures=concat_summarizing_dfs(MantelFolder)

In [ ]:
Mantels_newFeatures.sort_values(by='p_value')

In [ ]:
Mantels_newFeatures.sort_values(by='corr_coeff',ascending=False)

In [ ]:
Mantels_newFeatures[Mantels_newFeatures['phenotype_name'].str.contains('Mic')].sort_values(by='p_value')

In [ ]:
Mantels_newFeatures['feature_name'].unique()

In [ ]:
print len(Mantels_newFeatures['feature_name'].unique())

In [ ]:
featureList=Mantels_newFeatures['feature_name'].unique()

In [ ]:
for i, feature in enumerate(featureList):
    print i, feature
    if i==0:

        MantelOrganizedResults_newFeatures=Mantels_newFeatures[Mantels_newFeatures['feature_name']==feature]

    else:
        MantelOrganizedResults_newFeatures=pd.merge(MantelOrganizedResults_newFeatures,Mantels_newFeatures[Mantels_newFeatures['feature_name']==feature],
                                  how='outer', left_on=['phenotype_name','method','n_permut', 'alternative'],
                                               right_on=['phenotype_name','method','n_permut', 'alternative'],
                                    suffixes=('','_%s' %feature))


MantelOrganizedResults_newFeatures=MantelOrganizedResults_newFeatures.rename(columns={'corr_coeff':'corr_coeff_sharingMatrixMoreThan150PCsUsed_euclidean',
                                                    'p_value':'p_value_sharingMatrixMoreThan150PCsUsed_euclidean',
                                                    'N':'N_sharingMatrixMoreThan150PCsUsed_euclidean'})
for column in MantelOrganizedResults_newFeatures.columns.values:
        if 'feature_name' in column:
            MantelOrganizedResults_newFeatures=MantelOrganizedResults_newFeatures.drop(column,axis=1)
        newColName=column.replace('sharingMatrixMoreThan','')
        MantelOrganizedResults_newFeatures=MantelOrganizedResults_newFeatures.rename(columns={column:newColName})
                                                    
MantelOrganizedResults_newFeatures.head()       
    

In [ ]:
pList=[]
rList=[]

for column in MantelOrganizedResults_newFeatures.columns.values:
    if 'p_value' in column:
        pList.append(column)
    if 'corr_coeff' in column:
        rList.append(column)

MantelOrganizedResults_newFeatures['Mean p_value']=MantelOrganizedResults_newFeatures[pList].mean(axis=1)
MantelOrganizedResults_newFeatures['Mean corr_coeff']=MantelOrganizedResults_newFeatures[rList].mean(axis=1)
MantelOrganizedResults_newFeatures['Min p_value']=MantelOrganizedResults_newFeatures[pList].min(axis=1)
MantelOrganizedResults_newFeatures['Max corr_coeff']=MantelOrganizedResults_newFeatures[rList].max(axis=1)
MantelOrganizedResults_newFeatures.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults_newFeatures'
MantelOrganizedResults_newFeatures.to_pickle(file1)

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MantelOrganizedResults_newFeatures.xlsx'
MantelOrganizedResults_newFeatures.to_excel(file1)

# correlation plots

## develop correlation plot function to be incorporated into the mantel test function:

In [ ]:

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes/PNP434_Age_EucDistMat'
AgeDF=pd.read_pickle(file1)
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/PNP434_sharingMatrix_moreThan5_RA__braycurtis_distMat'
featureDF=pd.read_pickle(file2)

### process distance matrices:

In [ ]:
x=AgeDF
y=featureDF

print 'original x array shape is %s_%s' %(x.shape[0],x.shape[1])
print 'original y array shape is %s_%s' %(y.shape[0],y.shape[1])

for sample in x.index:
    if sample not in y.index:
        x=x.drop(sample,axis=0)
        x=x.drop(sample,axis=1)
for sample in y.index:
    if sample not in x.index:
        y=y.drop(sample,axis=0)
        y=y.drop(sample,axis=1)

for column in x:    #remove nonnumeric columns
    if x[column].dtype=='object':
        x=x.drop(column,axis=1)
        print '%s column was dropped from x' %column
x=x.dropna(axis=(0,1),how='all') #remove empty rows

for column in y:    #remove nonnumeric columns
    if y[column].dtype=='object':
        y=y.drop(column,axis=1)
        print '%s column was dropped from y' %column
y=y.dropna(axis=(0,1),how='all') #remove empty rows

print 'new x array shape is %s_%s' %(x.shape[0],x.shape[1])
print 'new y array shape is %s_%s' %(y.shape[0],y.shape[1])

In [ ]:
x=xValueList
y=yValueList

fig1=plt.figure(figsize=(6,6))

ymean=np.mean(y)                                                                                                                      

plt.scatter(x,y, alpha=0.1)
plt.xlabel('Age')
plt.ylabel('sharingMatrix_moreThan5_RA__braycurtis_distMat')



# #fit with np.polyfit
# nx=np.isnan(x)
# ny=np.isnan(y)
# n=nx+ny
# newx=list(x[~n])
# newy=list(y[~n])

# print newx
# print newy
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)),c='blue',linewidth=1)
# plt.plot(x=x,c='black', linewidth=4)
plt.title('Age to sharing_5RA distance matrix correlation')


from scipy.stats import pearsonr
r,p = pearsonr(x,y)


plt.annotate("r=%.4f p=%.6f" %(r,p),  xy=(0.02, 0.96), xycoords='axes fraction', fontsize=12,
    horizontalalignment='left', verticalalignment='top', fontweight='bold')



plt.show()

## The function: plot correlation scatter from two distance matrices:

In [ ]:
'''
x=feature matrix
y=phenotype matrix

x and y should be preprocessed to be at the same shape!

'''


def plot_corr_for_distMats(featureDistMat,PhenotypeDistMat,FeatureName,PhenotypeName,minPhenotypeValue):
    
    
    
    # generate lists of feature and phenotype distances for each sample combination:
    indList=[]
    xValueList=[]
    yValueList=[]

    for i,row in enumerate(featureDistMat.index):
        for j,column in enumerate(featureDistMat.columns.values):
            if row!=column:
                ind=row+'_'+column
                indList.append(ind)
                xValue=featureDistMat.loc[row,column]
                xValueList.append(xValue)
                yValue=PhenotypeDistMat.loc[row,column]
                yValueList.append(yValue)

    x=xValueList
    y=yValueList

    fig1=plt.figure(figsize=(8,8))

    ymean=np.mean(y)                                                                                                                      

    plt.scatter(x,y, alpha=0.1)
    plt.xlabel(FeatureName,fontsize=14)
    plt.ylabel(PhenotypeName,fontsize=14)
    plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)),c='blue',linewidth=1)
    plt.title('Distance matrix correlation',fontsize=16)
    
    from scipy.stats import pearsonr
    r,p = pearsonr(x,y)
    
    nSamples=len(featureDistMat)


    plt.annotate("r=%.4f p=%.6f, nSamples=%s" %(r,p,nSamples),  xy=(0.02, 0.96), xycoords='axes fraction', fontsize=14,
        horizontalalignment='left', verticalalignment='top', fontweight='bold')
    
    if minPhenotypeValue is not None:
        plt.ylim(minPhenotypeValue,np.max(y)*1.1)
#         plt.margins(0.2)
    
#     file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/DistMat_correlation_plots/'
#     fig1.savefig(file1,dpi=200)


    return fig1


    
    

    

In [ ]:
18*41

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes/PNP434_Total Protein_EucDistMat'
TotalProtDF=pd.read_pickle(file1)
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/PNP434_sharingMatrix_moreThan2_RA__braycurtis_distMat'
featureDF=pd.read_pickle(file2)

In [ ]:
x=TotalProtDF
y=featureDF

print 'original x array shape is %s_%s' %(x.shape[0],x.shape[1])
print 'original y array shape is %s_%s' %(y.shape[0],y.shape[1])

for sample in x.index:
    if sample not in y.index:
        x=x.drop(sample,axis=0)
        x=x.drop(sample,axis=1)
for sample in y.index:
    if sample not in x.index:
        y=y.drop(sample,axis=0)
        y=y.drop(sample,axis=1)

for column in x:    #remove nonnumeric columns
    if x[column].dtype=='object':
        x=x.drop(column,axis=1)
        print '%s column was dropped from x' %column
x=x.dropna(axis=(0,1),how='all') #remove empty rows

for column in y:    #remove nonnumeric columns
    if y[column].dtype=='object':
        y=y.drop(column,axis=1)
        print '%s column was dropped from y' %column
y=y.dropna(axis=(0,1),how='all') #remove empty rows

print 'new x array shape is %s_%s' %(x.shape[0],x.shape[1])
print 'new y array shape is %s_%s' %(y.shape[0],y.shape[1])

In [ ]:
FeatureName='sharingMatrix_moreThan2_RA'
PhenotypeName='TotalProtein'
fig1=plot_corr_for_distMats(y,x,FeatureName,PhenotypeName,minPhenotypeValue=0)
plt.show()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/DistMat_correlation_plots/%s_%s' %(FeatureName,PhenotypeName)
fig1.savefig(file1,dpi=200)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Phenotypes/PNP434_Age_EucDistMat'
TotalProtDF=pd.read_pickle(file1)
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/PNP434_sharingMatrix_moreThan5_RA__braycurtis_distMat'
featureDF=pd.read_pickle(file2)

In [ ]:
x=TotalProtDF
y=featureDF

print 'original x array shape is %s_%s' %(x.shape[0],x.shape[1])
print 'original y array shape is %s_%s' %(y.shape[0],y.shape[1])

for sample in x.index:
    if sample not in y.index:
        x=x.drop(sample,axis=0)
        x=x.drop(sample,axis=1)
for sample in y.index:
    if sample not in x.index:
        y=y.drop(sample,axis=0)
        y=y.drop(sample,axis=1)

for column in x:    #remove nonnumeric columns
    if x[column].dtype=='object':
        x=x.drop(column,axis=1)
        print '%s column was dropped from x' %column
x=x.dropna(axis=(0,1),how='all') #remove empty rows

for column in y:    #remove nonnumeric columns
    if y[column].dtype=='object':
        y=y.drop(column,axis=1)
        print '%s column was dropped from y' %column
y=y.dropna(axis=(0,1),how='all') #remove empty rows

print 'new x array shape is %s_%s' %(x.shape[0],x.shape[1])
print 'new y array shape is %s_%s' %(y.shape[0],y.shape[1])

In [ ]:
FeatureName='sharingMatrix_moreThan5_RA'
PhenotypeName='Age'
fig1=plot_corr_for_distMats(y,x,FeatureName,PhenotypeName,minPhenotypeValue=0)
plt.show()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/DistMat_correlation_plots/%s_%s' %(FeatureName,PhenotypeName)
fig1.savefig(file1,dpi=200)